In [1]:
import torch
from torch import load
import torch.nn.functional as F
from pathlib import Path
import os
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm

In [2]:
BS = 1
GPU = True
DIM = 300
SEARCH_BS = False

In [3]:
path = Path("../")
data_path = path/"data"
train_images_path = data_path / "train_images"
test_images_path = data_path / "test_images"
output_path = Path("./")
os.listdir(data_path)

['train_images',
 'test_images',
 'train_annotations',
 'train.csv',
 'sample_submission.csv',
 'test.csv',
 'submission.csv']

In [4]:
models_path = Path("../models")
os.listdir(models_path)

['02_unet_custom_DataParallel.pth',
 '02_unet_custom_DataParallel.zip',
 '08_Resnet18_finders_and_fastprogress.pth',
 '07_unet_resnet50_DataParallel.zip',
 '08_Resnet18_finders_and_fastprogress.zip',
 '07_unet_resnet50_DataParallel.pth',
 '01_unet_custom_model.pth',
 '08_unet_resnet50_DataParallel.zip',
 '01_unet_custom_weigths.pth',
 '08_unet_resnet50_DataParallel.pth']

# Dataframe

In [5]:
train_df = pd.read_csv(data_path / "train.csv")
train_df['path'] = train_df.id.apply(lambda x: f'{str(train_images_path)}/{x}.tiff')
train_df.head()

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex,path
0,10044,prostate,HPA,3000,3000,0.4,4,1459676 77 1462675 82 1465674 87 1468673 92 14...,37.0,Male,../data/train_images/10044.tiff
1,10274,prostate,HPA,3000,3000,0.4,4,715707 2 718705 8 721703 11 724701 18 727692 3...,76.0,Male,../data/train_images/10274.tiff
2,10392,spleen,HPA,3000,3000,0.4,4,1228631 20 1231629 24 1234624 40 1237623 47 12...,82.0,Male,../data/train_images/10392.tiff
3,10488,lung,HPA,3000,3000,0.4,4,3446519 15 3449517 17 3452514 20 3455510 24 34...,78.0,Male,../data/train_images/10488.tiff
4,10610,spleen,HPA,3000,3000,0.4,4,478925 68 481909 87 484893 105 487863 154 4908...,21.0,Female,../data/train_images/10610.tiff


# UNet Resnet18

In [6]:
number_of_classes = 2 # 2 classes
number_of_channels = 3

def conv3x3_bn(ci, co):
    return torch.nn.Sequential(
        torch.nn.Conv2d(ci, co, 3, padding=1),
        torch.nn.BatchNorm2d(co),
        torch.nn.ReLU(inplace=True)
    )

def encoder_conv(ci, co):
  return torch.nn.Sequential(
        torch.nn.MaxPool2d(2),
        conv3x3_bn(ci, co),
        conv3x3_bn(co, co),
    )

class deconv(torch.nn.Module):
    def __init__(self, ci, co):
        super(deconv, self).__init__()
        self.upsample = torch.nn.ConvTranspose2d(ci, co, 2, stride=2)
        self.conv1 = conv3x3_bn(ci, co)
        self.conv2 = conv3x3_bn(co, co)
    
    # recibe la salida de la capa anetrior y la salida de la etapa
    # correspondiente del encoder
    def forward(self, x1, x2):
        x1 = self.upsample(x1)
        diffX = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, (diffX, 0, diffY, 0))
        # concatenamos los tensores
        x = torch.cat([x2, x1], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class out_conv(torch.nn.Module):
    def __init__(self, ci, co, coo):
        super(out_conv, self).__init__()
        self.upsample = torch.nn.ConvTranspose2d(ci, co, 2, stride=2)
        self.conv = conv3x3_bn(ci, co)
        self.final = torch.nn.Conv2d(co, coo, 1)

    def forward(self, x1, x2):
        x1 = self.upsample(x1)
        diffX = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, (diffX, 0, diffY, 0))
        x = self.conv(x1)
        x = self.final(x)
        return x

class UNetResnet(torch.nn.Module):
    def __init__(self, n_classes=number_of_classes, in_ch=number_of_channels):
        super().__init__()

        self.encoder = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)           
        if in_ch != 3:
          self.encoder.conv1 = torch.nn.Conv2d(in_ch, 64, kernel_size=7, stride=2, padding=3, bias=False)

        self.deconv1 = deconv(512,256)
        self.deconv2 = deconv(256,128)
        self.deconv3 = deconv(128,64)
        self.out = out_conv(64, 64, n_classes)

    def forward(self, x):
        # x_in = torch.tensor(x.clone().detach())
        x_in = x.clone().detach()
        x = self.encoder.relu(self.encoder.bn1(self.encoder.conv1(x)))
        x1 = self.encoder.layer1(x)
        x2 = self.encoder.layer2(x1)
        x3 = self.encoder.layer3(x2)
        x = self.encoder.layer4(x3)
        x = self.deconv1(x, x3)
        x = self.deconv2(x, x2)
        x = self.deconv3(x, x1)
        x = self.out(x, x_in)
        return x

# Dice coeficient

In [7]:
def dice_coeff(pred, target):
    smooth = 1.
    num = pred.size(1)
    m1 = pred.view(num, -1).float()  # Flatten
    m2 = target.view(num, -1).float()  # Flatten
    intersection = (m1 * m2).sum().float()
    dice = (2. * intersection + smooth) / (m1.sum() + m2.sum() + smooth)
    dice = dice.item()

    return dice

# Functions

In [8]:
def rle2mask(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [
        np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])
    ]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    if len(shape) == 3:
        img = img.reshape(shape[0], shape[1])
    else:
        img = img.reshape(shape[0], shape[1])
    return img.T

def mask2rle(mask):
    '''
    mask: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def calc_mask(tensor, model):
    '''
    tensor: tensor of shape (1, 3, H, W)
    Returns mask of shape (H, W)
    '''
    if len(tensor.shape) == 3:
        tensor = tensor.unsqueeze(0)
    model.eval()
    with torch.no_grad():
        if GPU:
            tensor = tensor.to(device)
            model.to(device)
        else:
            print("CPU")
        output = model(tensor).squeeze(0)
        pred_mask = torch.argmax(output, axis=0)
    return pred_mask.detach().cpu().numpy()

def calc_mask_from_image_path(img_path, model):
    '''
    img_path: path to image
    model: model to use for inference
    Returns mask of shape (H, W)
    '''
    img = cv2.imread(img_path, cv2.COLOR_BGR2RGB) / 255.0
    img_tensor = torch.from_numpy(img.astype(np.float32)).float().permute(2, 0, 1).unsqueeze(0)
    return calc_mask(img_tensor, model)

def calc_rle_from_image_path(img_path, model):
    '''
    img_path: path to image
    model: model to use for inference
    Returns rle string
    '''
    mask = calc_mask_from_image_path(img_path, model)
    return mask2rle(mask)

# Dataset

In [9]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, dataframe, n_classes=2, dim=None, interpolation=cv2.INTER_LANCZOS4):
    self.dataframe = dataframe
    self.n_classes = n_classes
    self.dim = dim
    self.interpolation = interpolation

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, ix):
    # Get image path from column 'path' in dataframe
    img_path = str(self.dataframe.iloc[ix]['path'])
    # Load image
    img_cv = cv2.imread(img_path)
    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    # Resize image
    if self.dim is not None:
      if self.dim != 3000:
        img_cv_res = cv2.resize(img_cv, dsize=(self.dim, self.dim), interpolation=self.interpolation)
      else:
        img_cv_res = img_cv
    else:
      img_cv_res = img_cv
    # Normalize image
    img_cv_res_norm = img_cv_res / 255.0
    # Convert to tensor
    img_tensor = torch.from_numpy(img_cv_res_norm).float().permute(2, 0, 1)

    # Get image height and width
    img_height = int(self.dataframe.iloc[ix]['img_height'])
    img_width = int(self.dataframe.iloc[ix]['img_width'])
    
    return img_tensor, img_height, img_width

dataset = Dataset(train_df, dim = DIM)

# Load model

In [10]:
model_path = models_path / "07_unet_resnet50_DataParallel.pth"
with open(model_path, 'rb') as f:
    model = load(f)
    if GPU:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model.to(device)
if next(model.parameters()).is_cuda:
    print("Model load into GPU")
    GPU = True
else:
    print("Model load into CPU")
    GPU = False

Model load into GPU


# Inference function

In [11]:
def inference(model, dataloader, interpolation=cv2.INTER_LANCZOS4):
    criterion = torch.nn.BCEWithLogitsLoss()
    bar = tqdm(dataloader)
    model.eval()
    result = []
    scale = {'height':[],'width':[]}
    with torch.no_grad():
        for imgs, imgs_height, imgs_width in bar:
            # X and y to device
            imgs = imgs.to(device)

            # Compute prediction
            pred_masks = model(imgs)

            # Pass from one hot to tensor
            pred_masks = torch.argmax(pred_masks, axis=1).unsqueeze(0).type(torch.float32)

            scale['height'].extend(imgs_height)
            scale['width'].extend(imgs_width)
            result.extend(pred_masks.detach().cpu().numpy())
    return result, scale

# Batch size finder

In [12]:
if SEARCH_BS:
    def list_of_posible_batch_sizes(dataset):
        batch_sizes = []
        batch_size = 1
        while batch_size < 2*len(dataset):
            batch_sizes.append(batch_size)
            batch_size *= 2
        batch_sizes.sort(reverse=True)
        return batch_sizes

    BSs = list_of_posible_batch_sizes(dataset)
    BSs

In [13]:
if SEARCH_BS:
    for batchsize_find in BSs:
        print(f"batch size: {batchsize_find}")
        bs_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batchsize_find, shuffle=True, pin_memory=False, num_workers=4)
        epochs = 3
        out_of_memory = False
        for t in range(epochs):
            print(f"\tTrain epoch {t} of {epochs}")
            try:
                inference(model, bs_dataloader)
            except Exception as e:
                print(f'Error: {e}')
                out_of_memory = True
                break
        if out_of_memory == False:
            break
        print()
    print(f"Done!, bacth size is {batchsize_find}")

In [14]:
if SEARCH_BS:
    BS = batchsize_find

# Dataloader

In [15]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BS, shuffle=False, pin_memory=False, num_workers=4)

# Inference

In [16]:
result, scale = inference(model, dataloader)

100%|██████████| 351/351 [00:09<00:00, 36.56it/s]


In [17]:
result = np.array(result)
print(result.shape)
rle = []
for i,img in enumerate(result):
    print(img.shape, img.squeeze().shape, type(img), img.dtype)
    mask = cv2.resize(img, dsize=(int(scale['width'][i]),int(scale['height'][i])), fx = 1, fy = 1, interpolation = cv2.INTER_LINEAR)
    # img_c= cv2.resize(img_cv,        dsize=(self.dim, self.dim),                                             interpolation=self.interpolation)
    mask  = np.where(mask>0.5,1,0)
    rle.append(mask2rle(mask))

(351, 1, 300, 300)
(1, 300, 300) (300, 300) <class 'numpy.ndarray'> float32


: 

: 

In [ ]:
train_df['rle'] = rle
train_df.head()

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex,path
0,10044,prostate,HPA,3000,3000,0.4,4,0,37.0,Male,../data/train_images/10044.tiff
1,10274,prostate,HPA,3000,3000,0.4,4,1,76.0,Male,../data/train_images/10274.tiff
2,10392,spleen,HPA,3000,3000,0.4,4,2,82.0,Male,../data/train_images/10392.tiff
3,10488,lung,HPA,3000,3000,0.4,4,3,78.0,Male,../data/train_images/10488.tiff
4,10610,spleen,HPA,3000,3000,0.4,4,4,21.0,Female,../data/train_images/10610.tiff


In [ ]:
submission_df = train_df[['id', 'rle']]
submission_df.head()

,id,rle
0,10044,0
1,10274,1
2,10392,2
3,10488,3
4,10610,4


In [ ]:
submission_df.to_csv(output_path / "submission.csv", index=False)